In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pieces = []
for year in range(1880, 2011):
    path = f"datasets/babynames/yob{year}.txt"
    frame = pd.read_csv(path, names=["name", "sex", "births"])
 
    # 为年份添加一列
    frame["year"] = year
    pieces.append(frame)

# 将所有内容连接到单个DataFrame
names = pd.concat(pieces, ignore_index=True)

# 按性别分组生成两个DataFrame并重新排序索引
names_male = names[names['sex'] == 'M'].reset_index(drop=True)
names_female = names[names['sex'] == 'F'].reset_index(drop=True)

In [4]:
from sklearn.ensemble import RandomForestRegressor

def predict_top_names(df, year):
    # 获取输入年份前20年的数据
    df_last_20_years = df[df['year'].between(year-20, year-1)].copy()
    
    # 对数据进行预处理
    name_mapping = {name: code for code, name in enumerate(df_last_20_years['name'].unique())}
    df_last_20_years['name'] = df_last_20_years['name'].map(name_mapping)
    X = df_last_20_years[['name', 'year']]
    y = df_last_20_years['births']
    
    # 训练随机森林模型，使用所有可用的处理器
    model = RandomForestRegressor(n_estimators=100, n_jobs=-1)
    model.fit(X, y)
    
    # 预测指定年份的出生人数
    names_in_year = df[df['year'] == year]['name'].map(name_mapping).fillna(-1)
    X_test = pd.DataFrame({'name': names_in_year, 'year': year})
    y_pred = model.predict(X_test)
    
    # 获取预测出生人数最多的前10个名字
    top_10_indices = y_pred.argsort()[-10:]
    top_10_names = [df['name'][i] for i in top_10_indices]
    
    return top_10_names

while True:
    # 用户输入
    sex = input("请输入性别（男性输入'M'或'm'，女性输入'F'或'f'）：").upper()
    year = input("请输入年份（1900年到2010年）：")

    # 判断年份是否为数字
    if not year.isdigit():
        print("年份输入错误，请输入1900年到2010年之间的数字。")
        continue

    year = int(year)

    # 判断输入是否合法
    if sex not in ['M', 'F'] or year < 1900 or year > 2010:
        print("输入错误，请输入正确的性别和年份。")
        continue

    # 根据性别选择数据集
    if sex == 'M':
        df = names_male
    elif sex == 'F':
        df = names_female

    # 预测最受欢迎的前十个名字
    top_10_names = predict_top_names(df, year)
    print(f"{year}年预测的最受欢迎的前十个名字是：{top_10_names}")

    # 打印输入年份真实的最受欢迎的十个名字
    real_top_10_names = df[df['year'] == year]['name'].value_counts().index[:10].tolist()
    print(f"{year}年真实的最受欢迎的前十个名字是：{real_top_10_names}")
    break

请输入性别（男性输入'M'或'm'，女性输入'F'或'f'）：M
请输入年份（1900年到2010年）：2010
2010年预测的最受欢迎的前十个名字是：['Stuart', 'Houston', 'Al', 'Truman', 'Ted', 'Seth', 'Rolla', 'Wilford', 'John', 'William']
2010年真实的最受欢迎的前十个名字是：['Jacob', 'Saifan', 'Rosco', 'Rowin', 'Rowland', 'Royer', 'Ry', 'Rylynn', 'Saam', 'Saban']


# 朴素贝叶斯

In [5]:
from sklearn.naive_bayes import GaussianNB
import numpy as np

def predict_name_in_top(df, year, name):
    # 获取输入年份前20年的数据
    df_last_20_years = df[df['year'].between(year-20, year-1)].copy()
    
    # 对数据进行预处理
    name_mapping = {name: code for code, name in enumerate(df_last_20_years['name'].unique())}
    df_last_20_years['name'] = df_last_20_years['name'].map(name_mapping)
    
    # 创建目标变量，如果名字在最受欢迎的前五个名字中，则为1，否则为0
    df_last_20_years['in_top_5'] = df_last_20_years['name'].isin(df_last_20_years['name'].value_counts().index[:5]).astype(int)
    
    X = df_last_20_years[['name', 'year']].to_numpy()
    y = df_last_20_years['in_top_5'].to_numpy()
    
    # 训练朴素贝叶斯分类器
    model = GaussianNB()
    model.fit(X, y)
    
    # 预测指定名字在指定年份是否在最受欢迎的前五个名字中
    name_code = name_mapping.get(name, -1)
    X_test = np.array([[name_code, year]])
    pred = model.predict(X_test)
    
    return pred[0] == 1

# 用户输入
while True:
    sex = input("请输入性别（男性输入'M'，女性输入'F'）：").upper()
    if sex not in ['M', 'F']:
        print("输入错误，请输入'M'或'F'。")
        continue

    year = int(input("请输入年份（1900年到2010年）："))
    if year < 1900 or year > 2010:
        print("输入错误，年份应在1900年到2010年之间。")
        continue

    name = input("请输入名字：")

    # 根据性别选择数据集
    if sex == 'M':
        df = names_male
    elif sex == 'F':
        df = names_female

    # 预测名字是否在最受欢迎的前五个名字中
    is_in_top = predict_name_in_top(df, year, name)
    print(f"预测结果：在{year}年，名字{name}{'是' if is_in_top else '不是'}最受欢迎的前五个名字之一。")

    # 显示该年最受欢迎的五个名字
    top_5_names = df[df['year'] == year]['name'].value_counts().index[:5].tolist()
    print(f"{year}年最受欢迎的五个名字是：{top_5_names}")

    # 检测预测是否准确
    is_correct = is_in_top == (name in top_5_names)
    print(f"检测结果：预测{'准确' if is_correct else '不准确'}。")

    # 询问用户是否继续
    continue_choice = input("是否继续？（输入'Y'继续，其他键退出）：")
    if continue_choice.upper() != 'Y':
        break

请输入性别（男性输入'M'，女性输入'F'）：M
请输入年份（1900年到2010年）：2010
请输入名字：John
预测结果：在2010年，名字John不是最受欢迎的前五个名字之一。
2010年最受欢迎的五个名字是：['Jacob', 'Saifan', 'Rosco', 'Rowin', 'Rowland']
检测结果：预测准确。
是否继续？（输入'Y'继续，其他键退出）：
